In [1]:
import sys

sys.path.insert(0, "..")
sys.path.insert(0, "../disc_cop")
from disc_cop.envs import ENV_TO_FAMILY, ENVS

In [16]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import pandas as pd

from tqdm.notebook import tqdm

In [23]:
baseline_dir = "/Users/chanb/research/ualberta/Avg_OPE/datasets"

# result_dir = "/Users/chanb/research/ualberta/Avg_OPE/cc_results/results/disc_cop"
result_dir = "/Users/chanb/research/ualberta/Avg_OPE/cc_results/hyperparameter_tune/results"
env_name = "cartpole"

stats_file = "hyperparameter_tune.feather"

In [21]:
stats = []

for env_name in ENV_TO_FAMILY:
    mujoco = ENV_TO_FAMILY[env_name] == "mujoco"

    if mujoco:
        baseline_path = os.path.join(baseline_dir, "mujoco_obj.pkl")
    else:
        baseline_path = os.path.join(baseline_dir, "classic_obj.pkl")

    try:
        baselines = pickle.load(open(baseline_path, "rb"))[ENVS[ENV_TO_FAMILY[env_name]][env_name][0]]
    except:
        print("{} skipped".format(env_name))

    for run_file in tqdm(os.listdir(os.path.join(result_dir, env_name))):
        run_data = pickle.load(open(os.path.join(result_dir, env_name, run_file), "rb"))

        if len(run_data["results"]) == 0:
            print(run_file)
            continue

        gamma = run_data["hyperparameters"]["discount"]
        curr_baseline = baselines[
            [0.8, 0.9, 0.95, 0.99, 0.995].index(gamma)
        ]

        curr_res = []
        for seed in run_data["seeds"]:
            curr_res.append((np.array(run_data["results"][seed][1]) - curr_baseline) ** 2)

        stats.append(
            {
                "errors": curr_res,
                **run_data["hyperparameters"]
            }
        )

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

mse-tune-random_weight_2.0-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.001-alpha_0.1.pkl
mse-tune-random_weight_2.0-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.005-alpha_0.0.pkl
mse-tune-random_weight_2.0-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.005-alpha_0.1.pkl
mse-tune-random_weight_2.0-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.001-alpha_0.0.pkl
mse-tune-random_weight_2.0-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.005-alpha_0.01.pkl
mse-tune-random_weight_2.0-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.001-alpha_0.01.pkl
mse-tune-random_weight_2.0-discount_factor_0.95-max_ep_40-max_len_100-link_defau

...

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

mountain_car skipped


  0%|          | 0/24 [00:00<?, ?it/s]

mse-tune-random_weight_0.3-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.0005-alpha_0.1.pkl
mse-tune-random_weight_0.3-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.0001-alpha_0.0.pkl
mse-tune-random_weight_0.3-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.0001-alpha_0.1.pkl
mse-tune-random_weight_0.3-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.0005-alpha_0.0.pkl
mse-tune-random_weight_0.3-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_cross_q-lr_0.0005-alpha_0.01.pkl
mse-tune-random_weight_0.3-discount_factor_0.95-max_ep_40-max_len_100-link_default-batch_size_512-bootstrap_target_target_network-lr_0.0005-alpha_0.01.pkl
mse-tune-random_weight_0.3-discount_factor_0.95-max_ep_40-max_len_100-link_defaul

In [24]:
stats = pd.DataFrame(stats)
stats.to_feather(stats_file)

In [25]:
stats

,errors,lr,env,policy_path,link,random_weight,l1_lambda,checkpoint,epoch,cv_fold,batch_size,max_ep,max_len,use_batch_norm,use_target_network,discount,cop_discount,tau,save_path
0,"[[19.74001724399451, 11.525743643350763, 16.66...",0.001,Ant-v4,/home/chanb/src/Avg_OPE/exper/ant.pth,default,2.0,0.10,5,50000,1,512,40,100,False,True,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
1,"[[46.14155713133622, 5.674846784833404, 4.7658...",0.005,Ant-v4,/home/chanb/src/Avg_OPE/exper/ant.pth,default,2.0,0.00,5,50000,1,512,40,100,False,True,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
2,"[[50.662529617441685, 6.418897506078319, 5.129...",0.005,Ant-v4,/home/chanb/src/Avg_OPE/exper/ant.pth,default,2.0,0.10,5,50000,1,512,40,100,False,True,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
3,"[[19.271652493529896, 10.701166363337775, 16.7...",0.001,Ant-v4,/home/chanb/src/Avg_OPE/exper/ant.pth,default,2.0,0.00,5,50000,1,512,40,100,False,True,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
4,"[[47.736955960687126, 5.771766157294148, 4.879...",0.005,Ant-v4,/home/chanb/src/Avg_OPE/exper/ant.pth,default,2.0,0.01,5,50000,1,512,40,100,False,True,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,"[[58.34516988309245, 4.521298488955746, 13.507...",0.001,CartPole-v1,/home/chanb/src/Avg_OPE/exper/cartpole.pth,default,0.3,0.00,5,2000,1,512,40,100,False,True,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
140,"[[27.10121778531593, 26.70229230104881, 18.886...",0.005,CartPole-v1,/home/chanb/src/Avg_OPE/exper/cartpole.pth,default,0.3,0.01,5,2000,1,512,40,100,True,False,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
141,"[[37.32690236843865, 3.7611789145518517, 14.92...",0.001,CartPole-v1,/home/chanb/src/Avg_OPE/exper/cartpole.pth,default,0.3,0.01,5,2000,1,512,40,100,False,True,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...
142,"[[18.988125613231393, 20.30511474734324, 20.31...",0.001,CartPole-v1,/home/chanb/src/Avg_OPE/exper/cartpole.pth,default,0.3,0.01,5,2000,1,512,40,100,True,False,0.95,0.95,0.0005,/home/chanb/scratch/disc_cop/results/saved_mod...


In [29]:
stats.keys()

Index(['errors', 'lr', 'env', 'policy_path', 'link', 'random_weight',
       'l1_lambda', 'checkpoint', 'epoch', 'cv_fold', 'batch_size', 'max_ep',
       'max_len', 'use_batch_norm', 'use_target_network', 'discount',
       'cop_discount', 'tau', 'save_path'],
      dtype='object')

In [51]:
agg_res = {
    "mujoco": [[], []],
    "classic": [[], []],
}
for name, group in stats.groupby(["lr", "random_weight", "l1_lambda", "use_target_network"]):
    if name[1] == 0.3:
        key = "classic"
    else:
        key = "mujoco"
    agg_res[key][0].append(name)
    agg_res[key][1].append(np.array(group["errors"].to_list()).mean())

In [59]:
for family in ["classic", "mujoco"]:
    variants = np.array(agg_res[family][0])
    mse_per_variant = np.array(agg_res[family][1])
    sort_idxes = np.argsort(mse_per_variant)
    
    print(mse_per_variant[sort_idxes])

    print("BEST: {} {}".format(variants[sort_idxes[0]], mse_per_variant[sort_idxes[0]]))

[  5.05687929  13.13066058  14.19033043  15.61216855  15.89219325
  15.93477462  15.93847931  15.9775509   16.30915752  16.35080251
  16.44540474  16.63842158  17.11513371  18.22687569  18.29157837
  18.70861242  21.99609607  22.24248625  24.73796418  25.41881788
  26.2732119   52.01056295 101.65855866 126.85379437]
BEST: [0.005 0.3   0.    1.   ] 5.056879291494066
[3.43428830e+00 2.95940752e+01 1.89736070e+02 1.33033033e+06
 3.48861182e+06 2.73391470e+10 6.41141593e+12 1.80033612e+13
 3.77086585e+13 1.24281391e+14 4.11668016e+16 1.01203898e+17
 9.25696110e+17 9.63541387e+17 6.91909799e+18 7.00265309e+18
 9.82456050e+19 1.89464116e+20 1.98098449e+20 3.33400016e+27
            nan            nan            nan            nan]
BEST: [1.e-03 2.e+00 0.e+00 1.e+00] 3.4342882952295533
